In [2]:
import sys
import os
import pandas as pd
# import numpy as np
import numpy as np
from pathlib import Path
import sys
import os
import warnings
from pathlib import Path

# Get the absolute path of the current script
script_path = Path.cwd()
# Get the directory containing the script
src_dir = script_path.parent

sys.path.append(str(src_dir))


# Read data files

In [3]:
filtered_path = src_dir / 'data/OLD-sp-company-data'
filtered_name = 'filtered_ciqcompany'
filtered_norm_path = src_dir / 'data/OLD-sp-company-data-normalized'
filtered_norm_name = 'normalized'

In [4]:
from matcher.utils import read_files

col_names = ['companyid','companyname']
all_filtered_norm = read_files(filtered_norm_path, filtered_norm_name, col_names, sep='\t')
# all_filtered_source = read_files(filtered_path, filtered_name, col_names)
# all_filtered_source = all_filtered_source.astype({'companyid': 'int64'})


>> Num of files: 38


In [5]:
from matcher.core import SimCSE_Matcher
matcher = SimCSE_Matcher("sentence-transformers/all-MiniLM-L6-v2") 

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

In [6]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0


In [7]:
import pinecone      

pinecone.init(      
    api_key='4c1aef96-94be-4f75-8bec-b03058e896fa',      
    environment='us-west1-gcp'      
)      
index = pinecone.Index('inferess-name-links')

In [8]:
sample = all_filtered_norm[:10000]

In [10]:
embeddings = matcher.encode(sample['companyname'].tolist(), batch_size=64)

100%|██████████| 157/157 [00:03<00:00, 51.38it/s]


In [13]:
upsert_values = []
for i,row in sample.iterrows():
    print(i)
    break
        

0


In [ ]:
upsert_response = index.upsert(
    vectors=[
        {'id': 'vec1',
         'values': [0.1, 0.2, 0.3],
         'metadata': {'genre': 'drama'},
         'sparse_values': {
             'indices': [10, 45, 16],
             'values': [0.5, 0.5, 0.2]
         }},
        {'id': 'vec2',
         'values': [0.2, 0.3, 0.4],
         'metadata': {'genre': 'action'},
         'sparse_values': {
             'indices': [15, 40, 11],
             'values': [0.4, 0.5, 0.2]
         }}
    ],
    namespace='example-namespace'
)

Train a neural network to encode enities into dense dimension represent the charaters that match two entities# Create an dataset fot entity linking 

1. Collect positive links: 
`Collect highly scored names with the rule based methods that guarantee correctness of the matching`
---
2. Collect hard negative links:
`Hard negative matches means pretty close but not match to distinguish the distinct point that match between two names`
---
3. Symbol and what it stand for: 
`This hopefully will allow the neural networt to understand if some words can match it's intials`
---
4. Leverage the SME knowledge base to create some fake examples: 
`Searching for hard patterns like:`
- two words have no space between them
- some symbols used as short cut
- create fake shortcuts and real one as pos and neg samples



In [28]:
# !pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.0 MB/s eta 0:00:0000:01


In [29]:
from Levenshtein import distance, ratio

In [30]:

# %%time
# from tqdm import tqdm
# d = a
# similarities = np.zeros((d.shape[0], d.shape[0]))

# for i in tqdm(range(4314,len(d))):
#     for j in range(len(d)):
#         similarities[i][j] = ratio(a.iloc[i]['norm'], b.iloc[j]['norm'] )


  0%|          | 3/19322 [00:13<25:02:01,  4.66s/it]


KeyboardInterrupt: 

In [24]:
# np.save("data/sim2.npy", similarities[4314:15102])

In [ ]:
# neg_dict = {'word':[], 'neg':[]} 
# for i,sim in enumerate(similarities):
#     sorted = np.argsort(sim,axis=0)
#     top_sim = sorted[::-1][1:20]
#     top_sim = top_sim[np.where(sim[top_sim] >= 0.7)]
#     word = a.iloc[i]['name']
#     sim_neg = b.iloc[top_sim]['name'].tolist() 
#     neg_dict['word'] += [word]*len(sim_neg)

#     neg_dict['neg'] += sim_neg



# pd.DataFrame(neg_dict).to_json('data/neg_names.json')

In [32]:
# neg_name = pd.read_json('data/neg_names.json')

In [33]:
neg_name['norm'] = neg_name['word'].apply(lambda x :re.sub(r"(\w)([A-Z])", r"\1 \2", ' '.join([s.capitalize() if len(s) < 5 else s for s in x.split()])))\
    .apply(lambda x : x.lower()).apply(preprocess)

neg_name['norm_ws'] = neg_name['norm'].apply(lambda x : ntx.replace_term(x, ' ','') ) 

neg_name['neg_norm'] = neg_name['neg'].apply(lambda x :re.sub(r"(\w)([A-Z])", r"\1 \2", ' '.join([s.capitalize() if len(s) < 5 else s for s in x.split()])))\
    .apply(lambda x : x.lower()).apply(preprocess)

neg_name['neg_norm_ws'] = neg_name['neg_norm'].apply(lambda x : ntx.replace_term(x, ' ','') ) 


NameError: name 'neg_name' is not defined

In [32]:

positives = list(zip(a.norm_ws.tolist(),b.norm_ws.tolist()))
negatives = list(zip(neg_name.norm_ws.tolist(),neg_name.neg_norm_ws.tolist() ) )


X_train = positives + negatives
y_train = [1]*len(positives) + [0]*len(negatives)

assert len(X_train) == len(y_train)

pd.DataFrame({"X_train":X_train,'y_train':y_train} ).to_json('data/training_data_v1.json')



In [17]:
shuffle = True 
train_data = pd.read_json('data/training_data_v1.json')
if shuffle: 
    train_data = train_data.sample(frac=1)

In [18]:
train_data

,X_train,y_train
55971,"[bulova corp, lion corp]",0
3300,"[boston college, boston college]",1
42383,"[ariva ehf, artisan ehf]",0
28607,"[att inc, oath inc]",0
21216,"[toll bros inc, toll brors inc]",1
...,...,...
15318,"[oasis power llc, oasis power llc]",1
18740,"[saudi fooddrug authority, saudi fooddrug auth...",1
21223,"[tommy hilfiger corp, tommy hilfiger corp]",1
47718,"[bgc llc, vtc llc]",0


In [37]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

train_data.sample(20)

,X_train,y_train
35031,"[alleteinc, atexinc]",0
33542,"[airgasinc, arazuinc]",0
3712,"[cedromsniinc, cedromsniinc]",1
21695,"[upmcchildren'shospitalofpittsburgh, upmcchildren'shospitalofpittsburgh]",1
53328,"[bitzioinc, itroninc]",0
18194,"[sdc, sdc]",1
12760,"[mddsrl, mddsrl]",1
38964,"[amrepcorp, arielcorp]",0
6812,"[essabancorpinc, essabancorpinc]",1
11539,"[kochenergyservicesllc, kochenergyservicesllc]",1


In [21]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn.functional as F

# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
